In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset,labels):
    dataset=dataset.reshape((-1,image_size*image_size)).astype(np.float32)
    labels=(np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [9]:
train_subset = 1000

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset= tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights=tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    bias=tf.Variable(tf.zeros([num_labels]))
    
    logits= tf.matmul(tf_train_dataset,weights) + bias
    loss= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))
    
    optimizer=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + bias)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + bias)
    init=tf.global_variables_initializer()

In [13]:
num_steps = 801
def accuracy(predections,labels):
    return (100.0 *np.sum(np.argmax(predections,1) == np.argmax(labels,1))/predections.shape[0])


with tf.Session(graph=graph) as sess:
    sess.run(init)
    for num in  range(num_steps):
        _,l,predictions=sess.run([optimizer,loss,train_prediction])
        if (num % 100 == 0):
            print('Loss at step %d: %f' % (num, l))
            print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

Loss at step 0: 15.757600
Training accuracy: 16.7%
Validation accuracy: 14.6%
Loss at step 100: 1.279256
Training accuracy: 80.3%
Validation accuracy: 67.4%
Loss at step 200: 0.678344
Training accuracy: 86.6%
Validation accuracy: 69.2%
Loss at step 300: 0.400123
Training accuracy: 92.6%
Validation accuracy: 70.3%
Loss at step 400: 0.255150
Training accuracy: 95.2%
Validation accuracy: 70.4%
Loss at step 500: 0.167990
Training accuracy: 96.8%
Validation accuracy: 70.5%
Loss at step 600: 0.115398
Training accuracy: 98.8%
Validation accuracy: 70.5%
Loss at step 700: 0.084779
Training accuracy: 99.2%
Validation accuracy: 70.6%
Loss at step 800: 0.066998
Training accuracy: 99.5%
Validation accuracy: 70.8%
Test accuracy: 77.5%


In [16]:
batch_size = 128
num_hidden_nodes = 1024

graph=tf.Graph()
with graph.as_default():
    tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1=tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    biases1=tf.Variable(tf.zeros([num_hidden_nodes]))
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    lay1_train=tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
    logits=tf.matmul(lay1_train,weights2)+biases2
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)
    init=tf.global_variables_initializer()

In [17]:
num_steps=3001
with tf.Session(graph=graph) as sess:
    sess.run(init)
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict={tf_train_dataset: batch_data ,tf_train_labels: batch_labels}
        _,l,predictions = sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 353.581635
Minibatch accuracy: 10.2%
Validation accuracy: 33.5%
Minibatch loss at step 500: 7.357802
Minibatch accuracy: 83.6%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 11.795955
Minibatch accuracy: 75.8%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 5.261636
Minibatch accuracy: 78.9%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 5.624412
Minibatch accuracy: 78.1%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 5.215199
Minibatch accuracy: 78.1%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 6.067374
Minibatch accuracy: 75.8%
Validation accuracy: 81.7%
Test accuracy: 87.9%
